|||
|---|---|
|Title|nwtimetracking|
|Author|numbworks|
|Version|3.5.0|
||Please check [docs/docs-nwtimetracking.md](../docs/docs-nwtimetracking.md) before proceeding.|

#### Global Modules

In [129]:
from pandas import DataFrame

#### Local Modules

In [130]:
from nwtimetracking import SettingBag, DefaultPathProvider, YearProvider, TimeTrackingManager
from nwshared import VersionChecker, DisplayPreProcessor

#### Python Version Check

In [131]:
status : str = VersionChecker().get_python_version_status(required = (3, 12, 5))
print(status)

The installed Python version is matching the expected one (installed: '3.12.5', expected: '3.12.5').


#### Functions : Temp

#### Settings

In [132]:
setting_bag : SettingBag = SettingBag(
    years = YearProvider().get_all_years(),
    yearly_targets = YearProvider().get_all_yearly_targets(),
    excel_path = DefaultPathProvider().get_default_time_tracking_path(),
    excel_books_nrows = 1188,
    software_project_names = [
        "NW.MarkdownTables",
        "NW.NGramTextClassification",
        "NW.UnivariateForecasting",
        "NW.Shared.Files",
        "NW.Shared.Serialization",
        "NW.Shared.Validation",
        "nwreadinglist",
        "nwtimetracking",
        "nwtraderaanalytics",
        "nwshared"
	],
    software_project_names_by_spv = [
        "NW.MarkdownTables",
        "NW.NGramTextClassification",
        "NW.UnivariateForecasting",
        "NW.Shared.Files",        
        "NW.Shared.Serialization",
        "NW.Shared.Validation",
        "nwreadinglist",
        "nwtimetracking",
        "nwtraderaanalytics",
        "nwshared"
	], 
    tt_by_year_hashtag_years = [2024]
)


#### Main : Analysis

In [133]:
tt_manager : TimeTrackingManager = TimeTrackingManager()
sessions_df : DataFrame = tt_manager.get_sessions_dataset(setting_bag = setting_bag)
dpp : DisplayPreProcessor = DisplayPreProcessor()      

if setting_bag.show_sessions_df:
    display(sessions_df.tail(n = setting_bag.n_generic))


In [134]:
if setting_bag.show_tt_by_year_df:
    
    tt_by_year_df : DataFrame = tt_manager.get_tt_by_year(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        yearly_targets = setting_bag.yearly_targets)
    
    display(dpp.hide_index(df = tt_by_year_df))


Year,Effort,YearlyTarget,TargetDiff,IsTargetMet
2015,18h 00m,00h 00m,+18h 00m,True
2016,615h 15m,500h 00m,+115h 15m,True
2017,762h 45m,500h 00m,+262h 45m,True
2018,829h 45m,500h 00m,+329h 45m,True
2019,515h 15m,500h 00m,+15h 15m,True
2020,470h 30m,500h 00m,-30h 30m,False
2021,537h 30m,500h 00m,+37h 30m,True
2022,467h 30m,400h 00m,+67h 30m,True
2023,320h 15m,250h 00m,+70h 15m,True
2024,434h 30m,250h 00m,+184h 30m,True


In [135]:
if setting_bag.show_tt_by_year_month_df:

    tt_by_year_month_df : DataFrame = tt_manager.get_tt_by_year_month(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        yearly_targets = setting_bag.yearly_targets)
    
    display(tt_by_year_month_df.tail(n = setting_bag.n_generic))


,Year,Month,Effort,YearlyTotal,ToTarget
102,2024,5,43h 00m,273h 45m,+23h 45m
103,2024,6,48h 00m,321h 45m,+71h 45m
104,2024,7,67h 00m,388h 45m,+138h 45m
105,2024,8,32h 45m,421h 30m,+171h 30m
106,2024,9,13h 00m,434h 30m,+184h 30m


In [136]:
if setting_bag.show_tt_by_year_month_spnv_df:

    tt_by_year_month_spnv_df : DataFrame = tt_manager.get_tt_by_year_month_spnv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names)
    
    display(tt_by_year_month_spnv_df)

    tt_manager.try_print_definitions(df = tt_by_year_month_spnv_df, definitions = setting_bag.definitions)


In [137]:
if setting_bag.show_tt_by_year_spnv_df:

    tt_by_year_spnv_df : DataFrame = tt_manager.get_tt_by_year_spnv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names)
    
    display(tt_by_year_spnv_df)
    
    tt_manager.try_print_definitions(df = tt_by_year_spnv_df, definitions = setting_bag.definitions)


In [138]:
if setting_bag.show_tt_by_spn_df:

    tt_by_spn_df : DataFrame = tt_manager.get_tt_by_spn(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names,
        remove_untagged = setting_bag.remove_untagged_from_de
        )
    
    display(tt_by_spn_df)
    
    tt_manager.try_print_definitions(df = tt_by_spn_df, definitions = setting_bag.definitions)


,Hashtag,ProjectName,Effort,DE,%_DE,TE,%_TE
0,#python,nwtraderaanalytics,192h 15m,1701h 15m,11.30,2422h 30m,7.94
1,#python,nwreadinglist,97h 45m,1701h 15m,5.75,2422h 30m,4.04
2,#python,nwtimetracking,73h 00m,1701h 15m,4.29,2422h 30m,3.01
3,#python,nwshared,09h 30m,1701h 15m,0.56,2422h 30m,0.39
4,#csharp,NW.UnivariateForecasting,208h 00m,1701h 15m,12.23,2422h 30m,8.59
5,#csharp,NW.NGramTextClassification,207h 30m,1701h 15m,12.20,2422h 30m,8.57
6,#csharp,NW.MarkdownTables,20h 45m,1701h 15m,1.22,2422h 30m,0.86
7,#csharp,NW.Shared.Files,05h 30m,1701h 15m,0.32,2422h 30m,0.23
8,#csharp,NW.Shared.Serialization,04h 15m,1701h 15m,0.25,2422h 30m,0.18
9,#csharp,NW.Shared.Validation,02h 45m,1701h 15m,0.16,2422h 30m,0.11


DE: Development Effort
TE: Total Effort


In [139]:
if setting_bag.show_tt_by_spn_df:

    tt_by_spn_spv_df : DataFrame = tt_manager.get_tt_by_spn_spv(
        sessions_df = sessions_df, 
        years = setting_bag.years, 
        software_project_names = setting_bag.software_project_names_by_spv)
    
    display(dpp.hide_index(df = tt_by_spn_spv_df))


ProjectName,ProjectVersion,Effort
NW.MarkdownTables,1.0.0,15h 15m
NW.MarkdownTables,1.0.1,02h 30m
NW.MarkdownTables,3.0.0,03h 00m
NW.NGramTextClassification,1.0.0,73h 15m
NW.NGramTextClassification,1.1.0,07h 30m
NW.NGramTextClassification,2.0.0,15h 30m
NW.NGramTextClassification,3.0.0,28h 00m
NW.NGramTextClassification,3.5.0,39h 45m
NW.NGramTextClassification,3.6.0,17h 30m
NW.NGramTextClassification,3.7.0,08h 00m


In [140]:
if setting_bag.show_tt_by_year_hashtag:

    tt_by_year_hashtag_df : DataFrame = tt_manager.get_tt_by_year_hashtag(
        sessions_df = sessions_df, 
        years = setting_bag.tt_by_year_hashtag_years)
    
    display(dpp.hide_index(df = tt_by_year_hashtag_df))


Year,Hashtag,Effort
2024,#csharp,35h 15m
2024,#maintenance,92h 45m
2024,#python,227h 30m
2024,#studying,79h 00m


In [141]:
if setting_bag.show_tt_by_hashtag:
    tt_by_hashtag_df : DataFrame = tt_manager.get_tt_by_hashtag(sessions_df = sessions_df)
    display(tt_by_hashtag_df)


,Hashtag,Effort,Effort%
0,#untagged,2548h 45m,51.27
1,#csharp,1116h 45m,22.46
2,#python,440h 30m,8.86
3,#studying,373h 45m,7.52
4,#maintenance,318h 30m,6.41
5,#powershell,154h 00m,3.10
6,#overtime,19h 00m,0.38


In [142]:
if setting_bag.show_tts_by_month_df:
    
    tts_by_month_df : DataFrame = tt_manager.get_tts_by_month(sessions_df = sessions_df, years = setting_bag.years)

    if setting_bag.tts_by_month_update_future_values_to_empty:
        tts_by_month_upd_df : DataFrame = tt_manager.update_future_months_to_empty(tts_by_month_df = tts_by_month_df, now = setting_bag.now)
        display(dpp.hide_index(df = tts_by_month_upd_df))
    else:
        display(dpp.hide_index(df = tts_by_month_df))


Month,2015,↕,2016,↕,2017,↕,2018,↕,2019,↕,2020,↕,2021,↕,2022,↕,2023,↕,2024
1,00h 00m,↑,18h 00m,↑,88h 30m,↓,80h 15m,↓,60h 00m,↓,29h 15m,↑,53h 00m,↓,00h 00m,↑,06h 00m,↑,45h 45m
2,00h 00m,↑,45h 30m,↑,65h 30m,↑,90h 45m,↓,73h 00m,↓,38h 00m,↓,31h 30m,↓,03h 00m,↑,24h 00m,↑,77h 45m
3,00h 00m,↑,20h 45m,↑,71h 45m,↑,89h 00m,↓,75h 30m,↓,35h 00m,↑,40h 30m,↓,06h 15m,↑,50h 15m,↑,77h 45m
4,00h 00m,↑,37h 30m,↑,68h 00m,↑,88h 30m,↓,59h 45m,↓,40h 45m,↓,19h 00m,↑,27h 30m,↓,19h 00m,↑,29h 30m
5,00h 00m,↑,53h 00m,↑,83h 00m,↑,91h 15m,↓,54h 45m,↓,14h 30m,↑,112h 45m,↓,49h 45m,↓,31h 00m,↑,43h 00m
6,00h 00m,↑,57h 45m,↓,37h 45m,↑,62h 00m,↓,29h 15m,↓,12h 00m,↑,54h 00m,↑,73h 30m,↓,24h 45m,↑,48h 00m
7,00h 00m,↑,46h 45m,↑,65h 30m,↑,69h 30m,↓,24h 15m,↑,34h 00m,↓,23h 30m,↑,51h 00m,↓,16h 30m,↑,67h 00m
8,00h 00m,↑,25h 45m,↑,45h 45m,↑,72h 00m,↓,06h 00m,↑,32h 00m,↑,110h 00m,↓,36h 30m,↑,41h 30m,↓,32h 45m
9,00h 00m,↑,89h 30m,↓,43h 45m,↑,64h 00m,↓,39h 00m,↑,44h 00m,↓,43h 30m,↑,69h 00m,↓,50h 15m,↓,13h 00m
10,08h 00m,↑,82h 15m,↓,64h 30m,↓,46h 45m,↓,45h 30m,↑,48h 00m,↓,35h 30m,↑,38h 30m,↓,20h 00m,,


In [143]:
if (setting_bag.show_effort_status_df):
    
    es_df : DataFrame = tt_manager.add_effort_status(sessions_df = sessions_df)
    es_df = tt_manager.filter_by_is_correct(es_df = es_df, is_correct = setting_bag.effort_status_is_correct)
    
    if es_df.empty:
        display(es_df)
    else:
        display(es_df.head(n = setting_bag.effort_status_n))


,StartTime,EndTime,Effort,ES_IsCorrect,ES_Expected,ES_Message


In [144]:
if setting_bag.show_time_ranges_df:

    time_ranges_df : DataFrame = tt_manager.create_time_ranges_df(
        sessions_df = sessions_df, 
        unknown_id = setting_bag.time_ranges_unknown_id)
    
    if setting_bag.show_time_ranges_df:
        time_ranges_df = tt_manager.remove_unknown_id(
            time_ranges_df = time_ranges_df, 
            unknown_id = setting_bag.time_ranges_unknown_id)
        
    if setting_bag.time_ranges_filter_by_top_n:
        time_ranges_df = tt_manager.filter_by_top_n_occurrences(
            time_ranges_df = time_ranges_df, 
            n = setting_bag.time_ranges_top_n)

    display(dpp.hide_index(df = time_ranges_df))


TimeRangeId,Occurrences
08:00-08:45,32
08:00-08:30,24
18:00-20:00,20
19:00-20:00,16
17:30-18:00,16
